In [4]:
import gr00t
from gr00t.policy.gr00t_policy import Gr00tPolicy
from custom_modality_config import *
from gr00t.data.types import EmbodimentTag
import torch

/workspace/gr00t/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
device = "cpu"

In [7]:
# register_modality_config(my_config, embodiment_tag=EmbodimentTag.NEW_EMBODIMENT)

In [8]:
device

'cpu'

In [9]:
policy = Gr00tPolicy(embodiment_tag=EmbodimentTag.NEW_EMBODIMENT, model_path="nvidia/GR00T-N1.6-3B", device=device)#, modality_configs=my_config)

/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/workspace/gr00t/.venv/lib/python3.10/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 2.0.8 (you have 1.4.18). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


Tune backbone llm: False
Tune backbone visual: False
Backbone trainable parameter: model.language_model.model.layers.12.self_attn.q_proj.weight
Backbone trainable parameter: model.language_model.model.layers.12.self_attn.k_proj.weight
Backbone trainable parameter: model.language_model.model.layers.12.self_attn.v_proj.weight
Backbone trainable parameter: model.language_model.model.layers.12.self_attn.o_proj.weight
Backbone trainable parameter: model.language_model.model.layers.12.self_attn.q_norm.weight
Backbone trainable parameter: model.language_model.model.layers.12.self_attn.k_norm.weight
Backbone trainable parameter: model.language_model.model.layers.12.mlp.gate_proj.weight
Backbone trainable parameter: model.language_model.model.layers.12.mlp.up_proj.weight
Backbone trainable parameter: model.language_model.model.layers.12.mlp.down_proj.weight
Backbone trainable parameter: model.language_model.model.layers.12.input_layernorm.weight
Backbone trainable parameter: model.language_mode

/workspace/gr00t/gr00t/model/modules/dit.py:205: FutureWarning: Accessing config attribute `compute_dtype` directly via 'AlternateVLDiT' object attribute is deprecated. Please access 'compute_dtype' over 'AlternateVLDiT's config object instead, e.g. 'unet.config.compute_dtype'.
  embedding_dim=self.inner_dim, compute_dtype=self.compute_dtype
/workspace/gr00t/gr00t/model/modules/dit.py:236: FutureWarning: Accessing config attribute `output_dim` directly via 'AlternateVLDiT' object attribute is deprecated. Please access 'output_dim' over 'AlternateVLDiT's config object instead, e.g. 'unet.config.output_dim'.
  self.proj_out_2 = nn.Linear(self.inner_dim, self.output_dim)
`use_fast` is set to `True` but the image processor class does not have a fast version.  Falling back to the slow version.


Tune action head projector: True
Tune action head diffusion model: True
Tune action head vlln: True


Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  3.04it/s]


KeyError: 'new_embodiment'

In [12]:
from gr00t.model.gr00t_n1d6.gr00t_n1d6 import Gr00tN1d6, Gr00tN1d6Config

In [14]:
model_cfg = Gr00tN1d6Config()

In [15]:
model = Gr00tN1d6(config=model_cfg)

NameError: name 'Siglip2Model' is not defined

In [16]:
from gr00t.configs.finetune_config import FinetuneConfig

In [18]:
config = FinetuneConfig(
    base_model_path="nvidia/GR00T-N1.6-3B",
    dataset_path="/root/.cache/huggingface/lerobot/LSY-lab/stack_without_ft_tact_v4",
    embodiment_tag=EmbodimentTag.NEW_EMBODIMENT,
)

In [21]:
config.model

AttributeError: 'FinetuneConfig' object has no attribute 'model'

In [19]:
from gr00t.model import MODEL_REGISTRY


In [22]:
MODEL_REGISTRY

{gr00t.configs.model.gr00t_n1d6.Gr00tN1d6Config: gr00t.model.gr00t_n1d6.setup.Gr00tN1d6Pipeline}

In [20]:
pipeline = MODEL_REGISTRY.get(type(config.model))(config, "data")


AttributeError: 'FinetuneConfig' object has no attribute 'model'

In [23]:
from gr00t.experiment.launch_finetune import *

In [25]:
cfg = get_default_config()

In [26]:
cfg.model

Gr00tN1d6Config(model_type='Gr00tN1d6', model_dtype='bfloat16', model_name='nvidia/Eagle-Block2A-2B-v2', backbone_model_type='eagle', model_revision=None, tune_top_llm_layers=4, backbone_embedding_dim=2048, tune_llm=False, tune_visual=False, select_layer=16, reproject_vision=False, use_flash_attention=True, load_bf16=True, collator_overwrite_image_inputs=False, eagle_collator=False, backbone_trainable_params_fp32=True, image_crop_size=None, image_target_size=None, shortest_image_edge=256, crop_fraction=0.95, random_rotation_angle=None, color_jitter_params=None, use_albumentations_transforms=True, formalize_language=True, apply_sincos_state_encoding=False, use_relative_action=False, max_state_dim=29, max_action_dim=29, action_horizon=16, hidden_size=1024, input_embedding_dim=1536, add_pos_embed=True, attn_dropout=0.2, use_vlln=True, max_seq_len=1024, use_alternate_vl_dit=True, attend_text_every_n_blocks=2, diffusion_model_cfg={'positional_embeddings': None, 'num_layers': 32, 'num_attent